In [37]:
from search_engine.search_engine import SearchEngine
import networkx as nx
from networkx import Graph
from sklearn.model_selection import train_test_split
import implicit
import scipy.sparse as sparse
from scipy.sparse import csr_matrix
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import math
import igraph
import warnings
import csv
warnings.filterwarnings('ignore')

In [38]:
search_engine = SearchEngine()

Collection time: 0.0


In [39]:
search_engine.tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   created_at               10000 non-null  object
 1   id_str                   10000 non-null  object
 2   text                     10000 non-null  object
 3   source                   10000 non-null  object
 4   truncated                10000 non-null  bool  
 5   in_reply_to_screen_name  1564 non-null   object
 6   user                     10000 non-null  object
 7   retweeted_status         6933 non-null   object
 8   reply_count              10000 non-null  int64 
 9   retweet_count            10000 non-null  int64 
 10  favorite_count           10000 non-null  int64 
 11  entities                 10000 non-null  object
 12  favorited                10000 non-null  bool  
 13  retweeted                10000 non-null  bool  
 14  filter_level             10000 non-null

In [40]:
#u -> search_engine.tweets.user[0]['screen_name']
#v -> search_engine.tweets.retweeted_status[1]['user']['screen_name']

In [41]:
#Para Poner bonito -> dictionario de los usuarios v
#Mirar página en favoritos, lo del color también en la NS_P09
#Add weight edges from how many times user u retweeted user v

#g = nx.Graph()
#for tweet in search_engine.tweets.iterrows():
#    if str(tweet[1]['retweeted_status'])!='nan':
#        u=tweet[1]['user']['screen_name']
#        v=tweet[1]['retweeted_status']['user']['screen_name']
#        g.add_node(u)
#        g.add_node(v)
#        g.add_edge(u,v)

g=igraph.Graph()
for tweet in search_engine.tweets.iterrows():
    if str(tweet[1]['retweeted_status'])!='nan':
        u=tweet[1]['user']['screen_name']
        v=tweet[1]['retweeted_status']['user']['screen_name']
        g.add_vertices(u)
        g.add_vertices(v)
        g.add_edges([(u,v)])

In [42]:
#pos=nx.spring_layout(g)
#plt.figure(figsize=(15,15))
#nx.draw_networkx(g, pos, node_shape='o', node_size=50, with_labels=False, font_weight='bold')
#plt.show()

#layout = g.layout("kk")
#igraph.plot(g, layout = layout, target="/other-outputs/igraph.png")

In [66]:
print(g)

IGRAPH UN-- 13866 6933 --
+ attr: name (v)
+ edges (vertex names):
Liensevi--DGPurser, TrumpetteMAGA--KelemenCari, marigold33333--garynic67,
rosesantelli--MilionMagaMarch, w1iJhemeXEQy7RB--Breaking911,
VegasKatyin--UPTOWNGIRL2008, vonbernart--kylegriffin1,
Deb90243593--MondaireJones, Welding2KAG--JennaEllisEsq,
FalseFlagJoe1--RitaPanahi, word_criminal--LouDobbs, LouDobbs--fullofshitzu,
tweetontwitta--kylenabecker, SkizzGWK--ksorbs, JudithMorin58--catturd2,
Shona94724177--cjtruth, AlaskanGirl72--Barnes_Law, jdemboyz--davidsirota,
denis_horan--nycjim, joycee128--glennkirschner2, AkuaAgyemang0--PapaPoku,
grbsmith--mmpadellan, Lavy02--DewDropInn222, ronathepotato--marceelias,
Maryk53--AdamTexDavis, threadkillerksu--gatewaypundit,
ChaosKing6321--oneunderscore__, LauraThePatriot--CalebJHull,
KelemenCari--AliceV42035735, MikeDSpangler--CiceroConsulta1,
TRUMPMOMENTS411--JosephJFlynn1, KelemenCari--suburbprophet, RedX424--eScarry,
SolomanKarl--donwinslow, PapaPoku--Nanapercy_, sharronloder--Rud

*Recomender Systems*

In [43]:
#SELCET USER ID:
user_id=0

In [69]:
def find_nodes_at_distance_2(graph):

    all_potential_recommendations = set()
    
    for n1 in graph.vs:
        
        # all the nodes at distance 1
        nodes_at_most_distant_1 = set(graph.neighborhood(n1, order=1))

        # all the nodes at distance 1 and distance 2

        nodes_at_most_distant_2 = set(graph.neighborhood(n1, order=2))

        # only the nodes at distance 2
        only_nodes_at_distance_2 = nodes_at_most_distant_2 - nodes_at_most_distant_1
        
        
        # check if empty set
        if len(nodes_at_most_distant_2) > 0:
            for n2 in nodes_at_most_distant_2:
                
                # since n1 is an igraph vertex object, we need to extract the id
                n1_index = n1.index
                if n1_index!=n2:
                    all_potential_recommendations.add((n1_index, n2))
            
    return all_potential_recommendations

def predict_ALS(testset, model):
    # initialize the empty list
    all_predictions = []

    # scroll the obs
    for n1,n2, w in testset:
        
        # take here the low-dimensional vectors returned by the matrix factorization
        print(n1)
        array_n1 = model.user_factors[n1,:]
        array_n2 = model.item_factors[n2,:]

        # multiplying these vectors we generate an approximation for the edge score
        one_p = np.dot(array_n1, array_n2)

        all_predictions.append(one_p)
        
    return all_predictions

In [45]:
edges_max_distance_2=find_nodes_at_distance_2(g)

# fraction of edges to select as test-set
p = 0.2

# graphsize
N = len(g.es)

# idxs of all the edges
all_idxs = range(N)

# sample idxs of edges through the function "choice"
test_idxs = np.random.choice(a=all_idxs, size=int(p*N),replace=False)

ALS (Alternate Least Squares) #VER SI SON LOS FRIENDS-OF-FRIENDS

In [46]:
def number_to_username(alg_list: list, g) -> list:
    alg_list_transformed=[]
    for i in range(len(alg_list)):
        name=g.vs[int(alg_list[i][0])]['name']
        new_tuple=(name, alg_list[i][1])
        alg_list_transformed.append(new_tuple)
    return alg_list_transformed

def recommend_users(user_id: int, G:csr_matrix, g: igraph.Graph, top: int = 10) -> list:
    ALS_recommended_users=model.recommend(user_id, G, top)
    #ALS_recommended_users_with_names=number_to_username(ALS_recommended_users, g)
    return ALS_recommended_users

In [47]:
ground_truth = set()
trainset = set()
for idx, one_edge in enumerate(g.es):
    # take n1 and n2 idx from one_edge, that is an igraph edge *object*
    n1 = one_edge.source
    n2 = one_edge.target
    if idx in test_idxs:
        ground_truth.add((n1, n2, 1))
    else:
        trainset.add((n1, n2, 1))
        
for rec in edges_max_distance_2:
    
    # add to ground truth also the potential nodes
    n1 = rec[0]
    n2 = rec[1]
    
    ground_truth.add((n1,n2,0))

In [48]:
G = g.get_adjacency().data
G = csr_matrix(G)

In [71]:
model = implicit.als.AlternatingLeastSquares(factors=10, iterations=5, calculate_training_loss=True)
model.fit(G)

In [72]:
# generate the predictions
df_test = pd.DataFrame(list(ground_truth), columns=["n1","n2", "edge"])

all_predictions = predict_ALS(df_test.values, model)

# add predictions to df
df_test["rating"] = all_predictions

# convert predictions to binary values: 0 don't add the edge, 1 add it.
df_test["rating"] = df_test["rating"].apply(lambda x: round(x))

1346
7966
11552
11582
8480
5598
2038
4816
6936
9756
11576
1696
12884
4402
3410
8284
7808
4178
220
1436
5098
4876
1880
7274
11936
3800
5163
6578
1464
646
9990
1908
7438
5096
1150
11070
10462
94
3978
196
744
3400
28
802
1210
6524
9614
89
6668
10182
48
3608
9866
2504
4552
9096
838
85
6668
10828
5306
3924
374
5306
12530
1718
4718
5180
1443
2841
1872
6594
13252
8410
11378
10892
7214
9334
11884
2100
9080
730
10022
1776
11048
604
1286
9376
1538
279
7786
8694
9350
1982
7984
226
84
1360
8920
366
5924
5514
12000
1632
3520
6344
10230
11046
9124
1312
5926
11420
10282
5258
594
4230
510
6610
3872
8740
907
13444
11394
9642
882
7606
2032
3452
5098
6288
3698
3584
933
5106
852
1035
3046
10992
110
380
10036
1378
4342
11450
5444
10480
6802
7910
2730
1618
8260
348
6120
1424
12900
94
7676
7486
10896
11966
7843
6706
4566
10012
6002
9796
8564
8430
10032
8874
11272
4148
3410
2855
12702
13440
6814
7784
1300
241
11776
8424
5106
1364
1052
2606
12436
10480
6526
8990
220
9350
2012
2778
7188
12740
5656
4442
4230
738

13016
13392
394
11094
3694
7146
11718
13698
2188
8618
5022
3786
11276
9962
5806
4564
13786
2254
8428
11686
7598
1486
6836
4450
7678
6194
13208
3324
6484
1902
5834
5364
7608
6252
5252
2390
11702
10344
8552
12320
3534
1518
10052
1426
6830
9326
11498
7846
4368
11598
9314
1342
12738
894
7714
12900
2618
1929
8224
2116
10264
7782
10052
6530
2022
7186
3196
7598
774
10294
8894
1936
8556
10710
7988
10704
381
2850
2256
8418
1312
1468
4178
10388
2536
7426
3558
8812
142
5556
12922
6112
2082
2996
3636
402
318
2599
10988
11236
9954
4342
1802
81
4850
9744
8852
9652
538
13592
2930
5322
5294
7434
794
3164
10896
2024
12043
7242
9362
3146
11540
12386
2328
10922
8034
6668
318
7604
4242
1898
3698
6428
4180
3144
8364
6374
8778
2024
8110
2384
8314
6782
1058
2570
6240
2554
712
13728
13490
126
1268
3826
418
320
196
3534
10814
834
1879
3374
6024
882
119
10442
1860
3472
12404
838
3300
11456
10072
2324
2284
2940
13370
884
7052
12428
12814
3226
6462
7870
9758
8250
7202
530
406
8520
5298
8008
8926
10674
8796
334
45

2108
3506
1652
13808
5322
1982
2376
10252
10690
10014
326
10456
5752
8186
325
626
512
2986
11508
39
3280
12810
3694
5648
854
8034
6316
5212
7604
10968
5080
3090
5150
5806
9468
3498
11440
5314
5988
1094
13558
9758
11336
7374
6782
9508
8952
1318
306
1854
13382
9398
2376
12286
13312
6068
1174
4718
3014
7784
67
3094
4720
2549
6570
2498
8280
6475
11350
2428
11122
1863
10902
3606
2550
1094
573
2022
6310
2310
3190
7246
5088
88
2366
1414
4420
11560
12228
4382
9288
4336
12888
6640
740
3520
11934
12828
5056
13488
196
8998
6180
694
12122
10298
6484
4720
3430
10735
3946
12650
6240
11872
1272
406
2826
10992
5276
4922
13054
1174
3926
8990
10862
4876
12056
4636
874
2090
7354
1094
4110
168
9688
2056
9096
6734
10938
9282
3056
4082
13632
12776
8556
924
7200
13572
7866
12668
10134
5386
574
3698
2960
1155
2740
7758
110
3632
6024
7018
5282
11316
5932
1612
376
9080
10372
13306
6632
6956
5924
336
1708
482
456
13032
9272
3546
9336
4390
1406
1210
7260
26
440
4652
7188
12604
12760
11970
1360
11252
2276
2846
695

12742
89
12270
13040
154
1434
6462
2462
11872
11914
2390
13812
282
6716
4788
13300
6256
1246
798
796
6256
1618
227
7341
10454
10092
671
5026
4548
3068
2137
7614
135
9316
11374
1206
3958
882
2668
6560
538
11024
2444
12130
7622
10450
4552
7982
7724
735
7504
4674
4830
7250
1968
989
12948
6350
12604
3121
2971
5354
4884
4046
960
2760
3732
88
894
4280
3048
11066
7382
12964
3520
4638
2256
1896
1920
4728
12884
970
7652
12530
7412
60
9732
3700
4636
13592
11430
12918
2878
8440
922
3334
7110
5454
5534
1990
5980
4382
984
2250
2574
11866
13140
8484
6022
5272
11344
412
47
1094
12332
7928
639
13634
1790
7592
2108
9936
3909
6672
1310
11088
6644
5132
13726
10628
1630
2114
1395
8007
3510
9034
12900
1137
9966
6336
9132
12660
508
13202
1028
1620
1662
12088
7006
5656
4424
1288
5271
11360
8564
7354
2460
4151
8552
4202
1142
1930
8132
4850
11018
9528
6190
8410
13072
1952
2436
2410
9866
5838
1838
5392
9170
7776
2174
582
5826
10590
11056
10946
7884
13464
3848
6716
9812
1484
11818
3332
7046
1772
7084
9632
3694
1

4086
1600
435
246
7714
8892
239
7498
6830
2250
6814
3546
9980
6722
2846
8760
1701
6834
908
56
8582
670
11232
2348
13000
712
29
7660
7396
7422
416
7940
210
763
11048
3410
2680
12848
12386
11715
1000
8336
8032
3767
8978
3282
9870
11576
3504
2098
5104
3872
8042
2654
4874
4878
4266
11374
508
1113
38
317
334
7082
12932
1326
988
3638
4392
1468
6052
1552
11666
9756
9798
28
1082
11246
8982
5454
98
10536
7146
2602
14
3452
836
1112
3902
4616
6542
1090
13498
3012
1680
1142
12332
4680
3470
6410
13186
508
4540
2780
4012
13504
4340
1318
10828
8204
800
7667
2782
2444
255
404
1896
10310
1912
10896
13042
11552
12820
8292
4436
1828
11021
12130
2782
9844
12188
5434
3584
12727
9248
4896
11348
5654
3566
12526
11654
7808
3108
2238
7286
1290
984
4242
5022
13582
4566
938
1950
8004
11046
9880
10878
3704
5284
5848
154
8102
1988
5788
12456
1988
5104
7504
3270
7578
4442
1382
3510
5016
3924
8306
13582
11640
4100
7504
938
2616
4148
5396
6836
8568
1372
12154
140
7154
7188
6684
2498
5440
1046
12786
13780
7371
2380
58

...

10020
5620
7652
12562
1828
264
8694
11844
2778
10770
1606
5150
582
13010
5252
5408
176
5422
11606
604
13210
12120
6
1652
7146
4238
13594
486
4420
5656
7176
5306
5822
10004
13806
7960
5764
3400
236
13
1060
5274
14
9248
7924
540
7843
2684
2680
1614
1752
8458
797
5016
4462
94
5268
6401
5064
858
8976
12166
9558
7318
13
2188
11718
6560
1618
1246
8674
7908
9296
11162
2676
3084
222
4193
12320
794
5018
77
7318
834
11814
2032
6350
2202
2500
8672
11710
5836
10140
9536
7744
2250
10352
8530
6120
4178
8990
1242
11404
712
1846
3316
6530
6728
12902
3154
5440
2137
2462
8934
2656
7312
3702
5434
2004
1238
5788
10230
13740
308
6524
3394
2290
8044
12056
574
4160
6114
1090
6978
852
282
6682
773
6358
452
6110
7936
6726
328
4800
883
348
4132
988
8866
178
66
13092
7846
4390
10040
11336
5646
5818
1155
7608
8022
10198
6462
1504
852
497
4190
6610
12364
6836
9288
4869
11718
4720
2424
12130
844
1890
13780
6526
2077
5100
8478
8132
12198
8702
394
2928
7960
12492
1872
1296
6184
4554
2892
10474
1192
5144
3520
2547
107

538
12166
429
12884
10052
2286
11212
5804
7936
47
2828
6
8394
6168
3044
38
4446
5392
3870
5918
11872
1094
9756
7090
2082
2495
6172
9122
2098
394
8040
33
3442
6726
5096
12556
8666
9112
4840
1046
304
11252
1382
19
1186
7782
4562
2550
3190
9942
155
6608
6608
8796
13326
2796
622
11964
6446
482
7782
6830
6368
844
5388
7208
10946
9622
978
748
4034
6586
11646
8204
9478
1860
6858
10372
256
870
6526
9946
2288
1738
11010
11030
10052
11552
4266
6180
3598
264
13072
12190
8800
13598
6582
5136
5792
10122
11940
1052
10344
4110
3410
9464
11336
1524
3522
13352
297
1828
2012
4398
2768
226
1116
4520
2284
4230
1606
13366
10382
601
8428
5706
8364
1226
10674
12846
3226
7882
7908
8204
11856
2100
6978
1832
13316
12515
12666
597
10196
8418
11208
1468
3400
1468
6922
5672
572
5044
7974
11336
3092
10900
8122
6252
4392
12506
1504
236
2846
11258
1090
13312
12020
11914
12404
7843
8508
6024
4436
7856
5964
2328
12770
7308
908
7204
10850
2944
8418
1760
5298
4540
3394
9990
85
8280
376
9034
538
12060
318
10244
4608
9882


1618
9512
4876
13496
947
8036
536
10182
2460
142
10178
5056
4104
7508
9620
5904
1164
7318
6622
4238
5110
13110
11420
2802
5322
11384
3028
1924
12828
222
5106
8914
13312
1024
2550
8919
2024
9444
6680
10324
437
11336
50
774
2602
6426
1739
3430
721
8564
444
11246
5977
5444
5748
8034
11652
366
11050
9296
3794
354
3618
6390
5132
1272
12542
1832
2310
12516
7014
4900
10814
257
12826
7826
7522
12
11598
5062
2882
134
9376
3206
1618
4891
12188
5064
7112
7064
257
8418
8698
7406
11908
6228
464
964
7356
12008
4458
318
7458
2289
10108
5598
6716
3767
6576
3408
11108
13442
7650
530
4206
7064
5256
1036
4884
1425
11764
7864
1340
6628
10228
985
11868
11530
257
6180
5588
4873
10786
5792
1902
4336
354
513
3988
10456
12866
444
11020
4778
524
3844
4556
2678
1488
3400
8044
2558
10092
6252
1680
3800
6310
13558
13186
9744
6226
11438
1936
5150
2924
320
5318
9132
2098
10896
1958
508
2920
11018
876
652
9746
2768
1122
3128
1382
1902
2524
5806
7294
11752
12038
2152
10590
7368
12660
6420
6722
10187
8410
11772
5394
63

606
13822
9740
5422
847
4788
60
8606
3056
11336
1828
10368
12374
877
12540
4070
10812
413
2062
9962
340
8892
13812
997
3902
51
3348
1836
1651
8106
3374
279
538
11866
7958
7866
1902
7988
2111
1888
11702
414
62
1470
4718
7424
8390
3350
1952
5232
2796
2952
11216
13370
13032
10534
4566
6862
5206
9362
626
12428
1872
157
9622
3510
7096
1185
11312
11682
4278
1670
320
2970
5276
9746
7928
4540
39
6728
7374
6256
7242
6387
4174
13246
11336
1376
10134
9124
1144
3554
7902
7300
7924
9464
2841
4788
4368
8852
4114
11434
7254
6168
797
2778
3362
8426
4544
4896
2152
7422
4584
2174
12526
11056
436
2744
4496
5178
6478
3376
11812
11888
33
240
4358
304
2310
8874
3534
9990
3360
9056
1418
11032
2656
1406
4100
6026
7348
470
4122
4502
7052
12848
7974
10460
2062
12414
1800
3090
1386
6622
3004
4896
2098
10230
2986
10396
1926
9426
5454
7394
458
8842
1902
985
429
11258
8520
4160
278
119
255
5090
6336
728
4664
2272
3162
6594
1082
9034
5680
17
1630
11378
10012
1546
552
7732
8560
8988
6380
11246
688
3374
318
1618
4242


9230
20
13031
842
2954
6336
824
12660
2272
11492
1968
8954
10968
11436
6790
7214
3218
11064
7058
2550
2550
303
486
12436
10814
7776
6798
8794
5716
1139
1788
10182
2656
1250
7012
2542
4768
482
1116
13
2152
1376
1480
5708
4874
7310
7192
6534
4830
9622
13410
9362
8406
2290
13594
3980
9474
14
4564
3870
6766
143
8934
10230
3630
2288
3158
802
5464
3694
7308
212
11636
12738
2236
7628
456
9468
10462
11032
9272
11360
1832
2878
7843
7052
3300
12516
9116
3860
5680
2602
2688
10690
154
10854
221
7966
8566
7348
3348
13138
5396
802
4318
9496
10904
13618
5654
3190
10230
2238
4490
12896
1092
4390
8012
3394
1190
4720
6256
6484
2768
538
37
6917
9478
6130
6828
11180
11372
11026
4390
308
11056
9372
1210
3488
134
9732
12270
2656
885
1800
3584
5746
4552
2954
884
958
3854
2408
2768
12780
13120
12866
234
8934
3660
3826
1176
9310
6654
10334
10946
5496
3296
12088
9400
348
2748
6054
12888
11934
1052
8224
13602
3330
9488
1346
196
8476
3694
9034
6460
5410
8594
1828
1850
1342
4876
9572
10634
406
3584
7800
13202
6028

In [51]:
df_test.head()

,n1,n2,edge,rating
0,1346,1384,0,0
1,7966,11818,0,0
2,11552,6580,0,0
3,11582,1860,0,0
4,8480,6030,0,0


In [52]:
# number of observations matched by the prediction
right_predictions = len(df_test[df_test.edge == df_test.rating])

# accuracy
right_predictions/len(df_test)

0.9871036731450209

In [53]:
ALS_recomdetaion_ids=recommend_users(user_id, G, g)
ALS_recomdetaion_names=number_to_username(ALS_recomdetaion_ids, g)
ALS_recomdetaion_ids

[(169, 9.818881e-06),
 (47, 8.847937e-06),
 (67, 8.608822e-06),
 (429, 6.5014647e-06),
 (3, 5.1549587e-06),
 (143, 3.939685e-06),
 (253, 3.5919513e-06),
 (239, 2.7385493e-06),
 (595, 2.4629699e-06),
 (1680, 2.346808e-06)]

DCG ALS

In [54]:
print(g.neighborhood(user_id, order=1))
print()
print(g.neighbors(2029))
print(g.neighbors(1))

[0, 1]

[]
[0, 272, 1286, 3246, 3666]


 ---------------------------------------------------------------------------------------------------------------------------

Adamic-Adar

In [55]:
def get_recommendation_AA(username: int, g:igraph.Graph) -> pd.DataFrame:
    #We only need to consider those verices at distance 2
    #We take those users at distance 1
    neighbors_1=set(g.neighborhood(username, order=1))
    #We take those users at distance 1 & 2
    neighbors_2=set(g.neighborhood(username, order=2))
    #We take only those nodes that are at distance 2
    neighbors_only_order_2=list(neighbors_2 - neighbors_1)
    
    adamic_avar_data=pd.DataFrame(columns=[username], index=neighbors_only_order_2)
    for user_y in neighbors_only_order_2:
        if username!=user_y:
            x_neighbors=set(g.neighbors(username))
            y_neighbors=set(g.neighborhood(user_y))
        same_neighbors=x_neighbors&y_neighbors
        aa_val=0
        for n in same_neighbors:
            num_neighbors=len(g.neighbors(n))
            aa_val+=(1/math.log(num_neighbors,10))
        adamic_avar_data[username][user_y]=aa_val
    top_n_recommendations_aa=adamic_avar_data[username].sort_values(ascending=False)
    aa_final_recommendation=pd.DataFrame(top_n_recommendations_aa)
    return aa_final_recommendation.head(10)

def AA_num_to_name(dataset: pd.DataFrame, g: igraph.Graph)->pd.DataFrame:
    old_indices=list(dataset.index)
    new_indices=[]
    #name_column=g.vs[dataset=columns[0]]['name']
    main_user_id=dataset.columns[0]
    main_user_name=g.vs[main_user_id]['name']
    for user_id in old_indices:
        name=g.vs[user_id]['name']
        new_indices.append(name)
    new_dataset=pd.DataFrame(dataset.values, columns=[main_user_name], index=new_indices)
    return new_dataset

In [56]:
AA_recommendation_ids=get_recommendation_AA(user_id, g)
AA_recommendation_names=AA_num_to_name(AA_recommendation_ids, g)
AA_recommendation_names

,Liensevi
N30Foll0w,1.43068
FloydStad,1.43068
kasmouse,1.43068
dennis91842840,1.43068


----------------------------------------------------------------------------------------------------------------------------

PageRank

In [57]:
def pagerank_clearer(pagerank_values: list, g: igraph.Graph) -> list:
    pagerank=[]
    for i in range(len(pagerank_values)):
            user=pagerank_values[i][0]
            name=g.vs[user]['name']
            val=float(pagerank_values[i][1])
            pagerank.append((name, val))
    return pagerank

def top_10_ids(pagerank_result: list, user_id: int)->list:
    pagerank_with_ids=[]
    
    #We only need to consider those verices at distance 2
    #We take those users at distance 1
    neighbors_1=set(g.neighborhood(user_id, order=1))
    #We take those users at distance 1 & 2
    neighbors_2=set(g.neighborhood(user_id, order=2))
    #We take only those nodes that are at distance 2
    neighbors_only_order_2=list(neighbors_2 - neighbors_1)
    
    for i in range(len(pagerank_result)):
        if i!=user_id and i in neighbors_only_order_2:
            pagerank_with_ids.append([i, pagerank_result[i]])
    pagerank_with_ids.sort(key = lambda x: x[1], reverse=True)
    pagerank_top_10_ids=pagerank_with_ids[0:10]
    return pagerank_top_10_ids

In [58]:
##This will give the values of the users normalized??
#Do list (id, punctuation)
PageRank_recommendations=g.personalized_pagerank(directed=True, reset_vertices=user_id)
PageRank_recommendation_ids=top_10_ids(PageRank_recommendations, user_id)
PageRank_recommendation_names=pagerank_clearer(PageRank_recommendation_ids, g)
PageRank_recommendation_names

[('kasmouse', 0.0807136386394075),
 ('dennis91842840', 0.06518142272913913),
 ('N30Foll0w', 0.06518142272913913),
 ('FloydStad', 0.06518142272913913)]

----------------------------------------------------------------------------------------------------------------------------

Node2vec

In [77]:
import argparse
import numpy as np
import networkx as nx
import node2vec
from gensim.models import Word2Vec

In [78]:
A = g.get_edgelist()
graph = nx.Graph(A) # In case your graph is directed

In [79]:
#help(node2vec.Node2Vec)

In [80]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = node2vec.Node2Vec(graph, dimensions=64, walk_length=2, num_walks=200, workers=4) #Use temp_folder for big graphs
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, 
                                                             #`dimensions` and `workers` are automatically passed (from the 
                                                             #Node2Vec constructor)

In [63]:
user_id_str=str(user_id)
node2vec_recommendation_ids=model.wv.most_similar(user_id_str)

In [64]:
node2vec_recommendation_names=number_to_username(node2vec_recommendation_ids, g)

In [65]:
node2vec_recommendation_names

[('FloydStad', 0.9992849826812744),
 ('N30Foll0w', 0.9992270469665527),
 ('dennis91842840', 0.9991434216499329),
 ('kasmouse', 0.9295623302459717),
 ('Alexbobby2262C', 0.6820331811904907),
 ('ZeitlowGal', 0.6716615557670593),
 ('BJohonson', 0.6669320464134216),
 ('raslady1', 0.6645399928092957),
 ('MAGA777999', 0.664025068283081),
 ('StevenS82419733', 0.6636382341384888)]

----------------------------------------------------------------------------------------------------------------------------

## DOC2VEC

We wanted to exploit the text feature using Doc2Vec to recommend tweets based on the content of it. This was an opportunity for us to compare Word2Vec and Doc2Vec and see the pros and cons of both of them. So in this case, the nodes are not users but tweets, and based in the content a a tweet, we propose 10 tweets (Doc2Vec uses cosine similarity to do the recommendation, as well as Word2Vec).

In [190]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

# Initialization of Doc2Vec
def initialize_doc2vec(tweets: pd.DataFrame):
    tweets_ = []
    i = 0
    # Preparing the data to be put in Doc2Vec
    for line in tweets["original_text"]:
        tokens = simple_preprocess(line)
        tweets_.append(TaggedDocument(tokens, [i]))
        i += 1
    #Train the data and return
    d2v_model = Doc2Vec(documents=tweets_, vector_size=100, window=2, min_count=1, negative = 0, workers=4)
    return d2v_model

In [191]:
d2v_model = initialize_doc2vec(search_engine.tweets)

In [201]:
# This function returns the top 10 similar tweets related to a given tweet
def tweet2vec_top10(input_: str, tweets: pd.DataFrame):
    #Infer the vector to be able to let Doc2Vec do Cosine similarity
    embedded_input= d2v_model.infer_vector(input_.split())
    #Perform cosine similarity and return the top 10
    recommendations = d2v_model.docvecs.most_similar([embedded_input])
    #Preparing a DataFrame to return the top 10, by putting in the first row the input
    recommended_tweets = [input_]
    cosine_similarities = [1]
    results = pd.DataFrame(columns = ["Tweet", "Similarity"])
    # Preparing the list of the top 10
    for position, cos_similarity in recommendations:
        recommended_tweets.append(tweets["original_text"][position])
        cosine_similarities.append(str(cos_similarity))
    # Putting the lists in the DataFrame and returning
    results["Tweet"] = recommended_tweets
    results["Similarity"] = cosine_similarities
    return results

In [202]:
pd.set_option('display.max_colwidth', -1)
test = tweet2vec_top10(search_engine.tweets["original_text"][0], search_engine.tweets)
test

,Tweet,Similarity
0,"RT @DGPurser: December 7, 1941, a day which will live in infamy. Japan woke a sleeping tiger. So has the left - cheaters, the deep state,…",1
1,RT @Boyd_2650: 🔴🔵This election is NOT OVER &amp; Joe Biden HAS NOT WON! And we patriots CANNOT STAND BY &amp; LET EVIL LIBS &amp; FOREIGN COUNTRIES TRY…,0.36826881766319275
2,RT @davidfrum: Newt Gingrich's former seat in US House of Representatives is now held by a black woman who advocates stricter gun laws. Gin…,0.3492737412452698
3,RT @AdamParkhomenko: It really is fucking incredible the way trump can get caught committing a crime on Saturday and the whole damn city of…,0.3406864404678345
4,This is what Trump meant when he said Build the Wall,0.3288267254829407
5,@CNN TRUMP WON TRUMP WON TRUMP WON TRUMP WON FOUR MORE YEARS FOUR MORE YEARS ❤️😎❤️😎❤️😎❤️😎❤️😎❤️😎❤️,0.3229094445705414
6,RT @SidneyPowell1: And BIG TECH &amp; #Facebook #Google #Twitter are all into suppressing our freedom of speech to challenge this outrageous #E…,0.31539732217788696
7,"RT @TomFitton: ELECTION CRISIS: Are @sendavidperdue, @KLoeffler and other Senators prepared to object to Electoral College electors from st…",0.31435340642929077
8,"RT @KelemenCari: Judge asks Powell if Trump wins Georgia, can he win the election. Powell answers ""Yes, he can."" Fraud cannot be allowed to…",0.3082136809825897
9,"RT @thehill: Over 1,500 attorneys sign letter slamming Trump legal team's efforts to overturn election results http…",0.30532968044281006
